### Import Dependencies

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PayloadSchemaType, PointStruct, SparseVectorParams, Document, Prefetch, FusionQuery
from qdrant_client import models

import pandas as pd
import openai
import fastembed

In [ ]:
qdrant_client = QdrantClient(url="http://localhost:6333")

### Create Qdrant collection for hybrid search

In [ ]:
qdrant_client.create_collection(
    collection_name="Amazon-items-collection-01-hybrid-search",
    vectors_config={
        "text-embedding-3-small": VectorParams(size=1536, distance=Distance.COSINE)
    },
    sparse_vectors_config={
        "bm25": SparseVectorParams(modifier=models.Modifier.IDF)
    }
)

In [ ]:
qdrant_client.create_payload_index(
    collection_name="Amazon-items-collection-01-hybrid-search",
    field_name="parent_asin",
    field_schema=PayloadSchemaType.KEYWORD
)

### Embedding Functions

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model,
    )
    return response.data[0].embedding

In [ ]:
def get_embeddings_batch(text_list, model="text-embedding-3-small", batch_size=100):
    
    if len(text_list) <= batch_size:
        response = openai.embeddings.create(input=text_list, model=model)
        return [embedding.embedding for embedding in response.data]
    
    all_embeddings = []
    counter = 1
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i + batch_size]
        response = openai.embeddings.create(input=batch, model=model)
        all_embeddings.extend([embedding.embedding for embedding in response.data])
        print(f"Processed {counter * batch_size} of {len(text_list)}")
        counter += 1
    
    return all_embeddings

### Process and Embed Amazon Items Data

In [ ]:
df_items = pd.read_json("../../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", lines=True)

In [ ]:
df_items.head()

In [ ]:
len(df_items)

In [ ]:
def preprocess_description(row):
    return f"{row['title']} {' '.join(row['features'])}"

In [ ]:
def extract_first_large_image(row):
    return row["images"][0].get("large", "")

In [ ]:
df_items["description"] = df_items.apply(preprocess_description, axis=1)
df_items["image"] = df_items.apply(extract_first_large_image, axis=1)

In [ ]:
data_to_embed = df_items[["description", "image", "rating_number", "price", "average_rating", "parent_asin"]].to_dict(orient="records")

In [ ]:
data_to_embed

In [ ]:
text_to_embed = [data["description"] for data in data_to_embed]

In [ ]:
text_to_embed

In [ ]:
embeddings = get_embeddings_batch(text_to_embed)

In [ ]:
len(embeddings)

In [ ]:
pointstructs = []
i = 1
for embedding, data in zip(embeddings, data_to_embed):
    pointstructs.append(
        PointStruct(
            id=i,
            vector={
                "text-embedding-3-small": embedding,
                "bm25": Document(
                    text=data["description"],
                    model="qdrant/bm25"
                )
            },
            payload=data
        )
    )
    i += 1

In [ ]:
pointstructs[0].vector

In [ ]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-01-hybrid-search",
    points=pointstructs[0:500],
    wait=True
)

In [ ]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-01-hybrid-search",
    points=pointstructs[500:],
    wait=True
)

### Hybrid Retrieval

In [ ]:
def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01-hybrid-search",
        prefetch=[
            Prefetch(
                query=query_embedding,
                using="text-embedding-3-small",
                limit=20
            ),
            Prefetch(
                query=Document(
                    text=query,
                    model="qdrant/bm25"
                ),
                using="bm25",
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }

In [ ]:
results = retrieve_data("Can I get some tablet?", qdrant_client, k=20)

In [ ]:
results